In [ ]:
# ---------------------------------------------------------
# [Step 0] 라이브러리 자동 설치 및 설정 (오류 해결)
# ---------------------------------------------------------
import sys
import subprocess

# dart_fss가 없으면 자동으로 설치하는 로직
try:
    import dart_fss as dart
except ImportError:
    print("📦 라이브러리가 없어서 설치 중입니다... (약 1분 소요)")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "dart-fss", "pandas", "matplotlib", "seaborn"])
    import dart_fss as dart

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정 (Colab)
!sudo apt-get install -y fonts-nanum > /dev/null 2>&1
!sudo fc-cache -fv > /dev/null 2>&1
!rm ~/.cache/matplotlib -rf > /dev/null 2>&1
plt.rc('font', family='NanumBarunGothic')

# API 인증키 설정 (수정됨: api_key_list -> api_key)
api_key = 'YOUR_API_KEY'
dart.set_api_key(api_key=api_key)

# ---------------------------------------------------------
# [Step 1] 기본 정보 설정 (Manual Data from Screenshots)
# ---------------------------------------------------------
target_companies = {
    '000100': {'name': '유한양행', 'product_ratio': 0.485, 'domestic_ratio': 0.794}, #
    '128940': {'name': '한미약품', 'product_ratio': 0.978, 'domestic_ratio': 0.925}  #
}

# ---------------------------------------------------------
# [Step 2] API로 2025년 3분기 누적 데이터 추출 (Hybrid Logic)
# ---------------------------------------------------------
df_list = []
print("\n📡 DART 서버에 2025년 3분기 보고서 요청 중...")

for code, info in target_companies.items():
    try:
        # 1. API 호출
        corp_list = dart.get_corp_list()
        corp = corp_list.find_by_stock_code(code)

        # 2025년 분기보고서 검색
        fs = corp.extract_fs(bgn_de='20250101', report_tp='quarter')

        # 포괄손익계산서(cis) 우선, 없으면 손익계산서(is)
        df_fs = fs['cis'] if 'cis' in fs else fs['is']

        # 2. '누적' 데이터 컬럼 찾기 (정교한 필터링)
        target_col = None
        for col in df_fs.columns:
            col_str = str(col)
            # 2025년이 포함되고 '누적'이거나 시작일이 1월1일인 컬럼
            if '2025' in col_str and ('누적' in col_str or '20250101' in col_str):
                target_col = col
                break

        # 3. 데이터 추출 (실패 시 안전장치 사용)
        if target_col:
            # 계정명 검색 함수
            def get_val(keyword):
                try:
                    val = df_fs[df_fs['account_nm'].str.contains(keyword, na=False)][target_col].iloc[0]
                    return float(str(val).replace(',', ''))
                except:
                    return 0.0

            revenue = get_val('매출')
            op_profit = get_val('영업이익')

            # 데이터 유효성 체크 (0원이면 API 오류로 간주)
            if revenue == 0: raise ValueError("매출 데이터 누락")

            print(f"✅ {info['name']}: API 추출 성공 (매출 {revenue/100000000:.0f}억)")

        else:
            raise ValueError("2025년 누적 컬럼 못 찾음")

    except Exception as e:
        # [Fallback] API 실패 시, 작성자님이 제공한 스크린샷/공시 기반 수기 데이터 사용
        print(f"⚠️ {info['name']} API 연동 지연 ({e}) -> 2025 3Q 공시 확정치로 대체합니다.")
        if info['name'] == '유한양행':
            revenue, op_profit = 1640600000000, 78300000000
        else:
            revenue, op_profit = 1114600000000, 169419000000

    # R&D 비용 추정 (매출 * 비율)
    rnd_ratio = 0.097 if info['name'] == '유한양행' else 0.152 #

    df_list.append({
        'Company': info['name'],
        'Revenue_3Q': revenue,
        'OP_3Q': op_profit,
        'RnD_Expense': revenue * rnd_ratio,
        'Product_Ratio': info['product_ratio'],
        'Domestic_Ratio': info['domestic_ratio']
    })

df = pd.DataFrame(df_list)

# ---------------------------------------------------------
# [Step 3] 스트레스 테스트 (약가 15% 인하)
# ---------------------------------------------------------
price_cut = 0.15

def stress_test(row):
    exposed_rev = row['Revenue_3Q'] * row['Product_Ratio'] * row['Domestic_Ratio']
    loss = exposed_rev * price_cut

    rev_new = row['Revenue_3Q'] - loss
    op_new = row['OP_3Q'] - loss

    rnd_burden_old = (row['RnD_Expense'] / row['Revenue_3Q']) * 100
    rnd_burden_new = (row['RnD_Expense'] / rev_new) * 100

    return loss, op_new, rnd_burden_old, rnd_burden_new

df[['Revenue_Loss', 'OP_New', 'RnD_Burden_Old', 'RnD_Burden_New']] = df.apply(
    lambda x: pd.Series(stress_test(x)), axis=1
)

# ---------------------------------------------------------
# [Step 4] 결과 시각화
# ---------------------------------------------------------
display_df = df.copy()
for c in ['Revenue_3Q', 'OP_3Q', 'Revenue_Loss', 'OP_New']:
    display_df[c] /= 100000000 # 억 단위

print("\n[📊 2025년 3분기 누적 기준 시뮬레이션 결과]")
display(display_df[['Company', 'Revenue_3Q', 'OP_3Q', 'Revenue_Loss', 'OP_New', 'RnD_Burden_New']].round(1))

plt.figure(figsize=(10, 6))
df_melt = df.melt(id_vars='Company', value_vars=['RnD_Burden_Old', 'RnD_Burden_New'], var_name='Scenario', value_name='Ratio')
bars = sns.barplot(x='Company', y='Ratio', hue='Scenario', data=df_melt, palette='coolwarm')

plt.title('약가 인하 시 R&D 비용 부담률 변화 (2025 3Q 누적)', fontsize=14, fontweight='bold')
plt.ylabel('매출액 대비 R&D 비중 (%)')
plt.ylim(0, 20)
plt.legend(title='시나리오', labels=['현재', '약가인하 후'])

for p in bars.patches:
    height = p.get_height()
    if height > 0:
        plt.text(p.get_x() + p.get_width() / 2., height + 0.2, f'{height:.1f}%', ha='center', fontweight='bold')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import pandas as pd # 데이터 처리를 위해 필요

# 1. 캔버스 준비
plt.figure(figsize=(12, 8)) # 세로 길이를 조금 더 늘림

# 2. 데이터 정리
df_op = df.melt(id_vars='Company', value_vars=['OP_3Q', 'OP_New'],
                var_name='Scenario', value_name='Operating Profit')

# 3. 막대 그래프 그리기
ax = sns.barplot(x='Company', y='Operating Profit', hue='Scenario',
                 data=df_op, palette='RdBu')

# 4. 제목 및 축 설정
plt.title('약가 인하 시 영업이익 증발 및 적자 전환 (Profit Impact)', fontsize=16, fontweight='bold')
plt.ylabel('영업이익 (억 원)', fontsize=12)
plt.axhline(0, color='black', linewidth=1.5) # 0원 기준선
plt.grid(axis='y', linestyle='--', alpha=0.5)

# [핵심 수정 1] Y축 범위(여백) 강제 확장
# 가장 낮은 값(적자)과 가장 높은 값(흑자)을 찾아서 위아래로 20%씩 공간을 더 줍니다.
y_min = df_op['Operating Profit'].min()
y_max = df_op['Operating Profit'].max()
# 아래로 20% 더 내리고, 위로 15% 더 올림 (글씨 공간 확보)
plt.ylim(y_min * 1.3, y_max * 1.15)

# 5. Y축 눈금 포맷터 (억 단위)
def billions(x, pos):
    return f'{int(x * 1e-8):,}억'
ax.yaxis.set_major_formatter(FuncFormatter(billions))

# 6. 막대 위에 숫자 표시 ([핵심 수정 2] 위치 잡는 방식 변경)
for p in ax.patches:
    height = p.get_height()

    # 막대가 없거나 데이터가 없는 경우 패스
    if height == 0 or pd.isna(height):
        continue

    # 값 변환 (원 -> 억)
    # 데이터가 이미 억 단위일 수도 있고 원 단위일 수도 있으니 안전하게 처리
    # (절대값이 10,000보다 크면 원 단위라고 간주)
    val_in_billion = height / 100000000 if abs(height) > 10000 else height

    # 텍스트 설정
    text = f'{int(val_in_billion):,}억'
    color = 'red' if val_in_billion < 0 else 'black'

    # [핵심 수정 3] 좌표 기반이 아니라 '오프셋(offset)' 방식으로 위치 지정
    # 이렇게 하면 데이터 단위가 뭐든 상관없이 무조건 막대 끝에서 10포인트 떨어진 곳에 찍힘
    xy_pos = (p.get_x() + p.get_width() / 2, height)

    if height < 0: # 음수(적자)인 경우
        xytext = (0, -15) # 막대 아래로 15포인트 이동
        va = 'top'        # 텍스트 상단을 기준점으로 (아래로 매달림)
    else:          # 양수(흑자)인 경우
        xytext = (0, 5)   # 막대 위로 5포인트 이동
        va = 'bottom'     # 텍스트 하단을 기준점으로 (위로 얹음)

    ax.annotate(text, xy=xy_pos, xytext=xytext, textcoords='offset points',
                ha='center', va=va, fontsize=13, fontweight='bold', color=color)

# 7. 범례 및 출력
plt.legend(title='시나리오', labels=['현재(Before)', '약가인하 후(After)'], loc='upper right')
plt.tight_layout()
plt.show()